# NData

In [1]:
#| default_exp ndata

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [21]:
#| export
import torch, inspect, numpy as np, scipy.sparse as sp
from typing import Callable, Optional, Union, Dict
from torch.utils.data import DataLoader
from transformers import BatchEncoding

from xcai.core import Filterer, Info
from xcai.data import _read_sparse_file
from xcai.sdata import identity_collate_fn
from xcai.sdata import SMainXCDataset, SXCDataset
from xcai.sdata import SBaseXCDataBlock, SXCDataBlock

from fastcore.utils import *
from fastcore.meta import *
from plum import dispatch

In [4]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

## Setup

In [22]:
from xcai.main import *

In [26]:
pkl_dir = '/Users/suchith720/Projects/data/processed/mogicX'
pkl_file = f'{pkl_dir}/wikiseealsotitles_data_distilbert-base-uncased_sxc.joblib'

data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
config_file = 'wikiseealsotitles'
config_key = 'data_meta'

In [27]:
block = build_block(pkl_file, config_file, True, config_key, data_dir=data_dir, n_slbl_samples=2, do_build=False, 
                    main_oversample=True, meta_oversample=True, return_scores=True)

## Data

In [6]:
#| export
class NMainXCData:
    
    @classmethod
    def from_file(cls, data_info:str, lbl_info:str, data_lbl:Optional[str]=None, data_neg:Optional[str]=None, 
                  data_lbl_filterer:Optional[str]=None, main_max_data_sequence_length:Optional[int]=None, 
                  main_max_lbl_sequence_length:Optional[int]=None, **kwargs):
        return {
            'data_lbl': _read_sparse_file(data_lbl),
            'data_neg': _read_sparse_file(data_neg),
            'data_info': Info.from_txt(data_info, max_sequence_length=main_max_data_sequence_length, **kwargs),
            'lbl_info': Info.from_txt(lbl_info, max_sequence_length=main_max_lbl_sequence_length, **kwargs),
            'data_lbl_filterer': Filterer.load_filter(data_lbl_filterer),
        }

## NDataset

### `SMainWithNegativesXCDataset`

In [8]:
#| export
class NMainXCDataset(SMainXCDataset):

    def __init__(
        self,
        data_neg:Optional[sp.csr_matrix]=None,
        **kwargs
    ):
        store_attr('data_neg')
        self.curr_data_neg, self.data_neg_scores = None, None
        super().__init__(**kwargs)

    def _store_indices(self):
        super()._store_indices()
        if self.data_neg is not None: self.curr_data_neg = [o.indices.tolist() for o in self.data_neg]

    def _verify_inputs(self):
        super()._verify_inputs()
        if self.data_neg is not None and self.data_lbl is not None:
            if self.data_neg.shape != self.data_lbl.shape:
                raise ValueError(f"Shape mismatch: `self.data_lbl`({self.data_lbl.shape}) and `self.data_neg`({self.data_neg.shape})")

    def _store_scores(self):
        super()._store_scores()
        if self.data_neg is not None:
            if self.use_main_distribution:
                data_neg = self.data_neg / (self.data_neg.sum(axis=1) + 1e-9)
                data_neg = data_neg.tocsr()
            else:
                data_neg = self.data_neg
            self.data_neg_scores = [o.data.tolist() for o in data_neg]
            
    def __getitems__(self, idxs:List):
        x = {'data_idx': torch.tensor(idxs, dtype=torch.int64)}
        x.update(self.get_info('data', idxs, self.data_info, self.data_info_keys))
        if self.data_lbl is not None:
            prefix = 'lbl2data'
            o = self.extract_items(prefix, self.curr_data_lbl, idxs, self.n_lbl_samples, self.n_slbl_samples, self.main_oversample, 
                                   self.lbl_info, self.lbl_info_keys, self.use_main_distribution, self.data_lbl_scores, 
                                   return_scores=self.return_scores)
            x.update(o)
            
        if self.data_neg is not None:
            prefix = 'neg2data'
            o = self.extract_items(prefix, self.curr_data_neg, idxs, self.n_lbl_samples, self.n_slbl_samples, self.main_oversample, 
                                   self.lbl_info, self.lbl_info_keys, self.use_main_distribution, self.data_neg_scores, 
                                   return_scores=self.return_scores)
            x.update(o)
            
        return x

    @classmethod
    @delegates(NMainXCData.from_file)
    def from_file(
        cls, 
        n_lbl_samples:Optional[int]=None,
        data_info_keys:Optional[List]=None,
        lbl_info_keys:Optional[List]=None,
        n_slbl_samples:Optional[int]=1,
        main_oversample:Optional[bool]=False,
        use_main_distribution:Optional[bool]=False,
        return_scores:Optional[bool]=False,
        **kwargs
    ):
        return cls(**NMainXCData.from_file(**kwargs), n_lbl_samples=n_lbl_samples, data_info_keys=data_info_keys,
                   lbl_info_keys=lbl_info_keys, n_slbl_samples=n_slbl_samples, main_oversample=main_oversample, 
                   use_main_distribution=use_main_distribution, return_scores=return_scores)

    def _getitems(cls, idxs:List):
        raise NotImplementedError('Function should be implemented in the subclass.')
    

### `NXCDataset`

In [13]:
#| export
class NXCDataset(SXCDataset):

    def __init__(self, data:NMainXCDataset, **kwargs):
        super().__init__()
        self.data, self.meta = data, MetaXCDatasets({k:kwargs[k] for k in self.get_meta_args(**kwargs) if isinstance(kwargs[k], SMetaXCDataset)})
        self._verify_inputs()

    @classmethod
    @delegates(SMainXCDataset.from_file)
    def from_file(cls, **kwargs):
        data = NMainXCDataset.from_file(**kwargs)
        meta_kwargs = {o:kwargs.pop(o) for o in cls.get_meta_args(**kwargs)}
        meta = {k:SMetaXCDataset.from_file(**v, **kwargs) for k,v in meta_kwargs.items()}
        return cls(data, **meta)
        

### `SBaseXCDataBlock`

In [19]:
#| export
class NBaseXCDataBlock(SBaseXCDataBlock):

    @delegates(DataLoader.__init__)
    def __init__(
        self, 
        dset:NXCDataset, 
        collate_fn:Optional[Callable]=identity_collate_fn,
        **kwargs
    ):
        self.dset, self.dl_kwargs, self.collate_fn = dset, self._get_dl_kwargs(**kwargs), collate_fn
        self.dl = DataLoader(dset, collate_fn=collate_fn, **self.dl_kwargs) if collate_fn is not None else None

    @classmethod
    @delegates(NXCDataset.from_file)
    def from_file(cls, collate_fn:Callable=identity_collate_fn, **kwargs):
        return NBaseXCDataBlock(NXCDataset.from_file(**kwargs), collate_fn, **kwargs)
        

### `NXCDataBlock`

In [22]:
#| export
class NXCDataBlock(SXCDataBlock):

    @classmethod
    def from_cfg(
        cls, 
        cfg:Union[str,Dict],
        collate_fn:Optional[Callable]=identity_collate_fn,
        valid_pct:Optional[float]=0.2,
        seed=None,
        **kwargs,
    ):
        if isinstance(cfg, str): cfg = cls.load_cfg(cfg)

        blocks = dict()
        for o in ['train', 'valid', 'test']:
            if o in cfg['path']:
                params = cfg['parameters'].copy()
                params.update(kwargs)
                if o != 'train': 
                    params['meta_dropout_remove'], params['meta_dropout_replace'] = None, None
                blocks[o] = NBaseXCDataBlock.from_file(**cfg['path'][o], **params, collate_fn=collate_fn)
                
        return cls(**blocks)
        